# A simple workflow to download geospatial imagery from a particular ROI and date range

> Daniel Buscombe, Feb 11 2021

Import libraries

In [1]:
import geemap
import ee
import os
from ipyleaflet import GeoJSON
import numpy as np

Inputs

In [8]:
start_date = '2015-01-01'
end_date = '2020-12-31'

MAXCLOUD = 5  #percentage maximum cloud cover tolerated

# import json bounding box from somewhere (or draw here:)
#https://geojson.io/#map=16/37.7380/-122.5056

## ocean beach
json_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.51575469970703,
              37.752665386359325
            ],
            [
              -122.50785827636719,
              37.752665386359325
            ],
            [
              -122.50785827636719,
              37.78007695280165
            ],
            [
              -122.51575469970703,
              37.78007695280165
            ],
            [
              -122.51575469970703,
              37.752665386359325
            ]
          ]
        ]
      }
    }
  ]
}

# funston
# json_data = {
#   "type": "FeatureCollection",
#   "features": [
#     {
#       "type": "Feature",
#       "properties": {},
#       "geometry": {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               -122.50974655151367,
#               37.702701373084956
#             ],
#             [
#               -122.5001335144043,
#               37.702701373084956
#             ],
#             [
#               -122.5001335144043,
#               37.7293168367123
#             ],
#             [
#               -122.50974655151367,
#               37.7293168367123
#             ],
#             [
#               -122.50974655151367,
#               37.702701373084956
#             ]
#           ]
#         ]
#       }
#     }
#   ]
# }

# #sunset state beach, 
# json_data ={
#   "type": "FeatureCollection",
#   "features": [
#     {
#       "type": "Feature",
#       "properties": {},
#       "geometry": {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               -121.83912992477416,
#               36.880204177187395
#             ],
#             [
#               -121.8283796310425,
#               36.880204177187395
#             ],
#             [
#               -121.8283796310425,
#               36.89489499497731
#             ],
#             [
#               -121.83912992477416,
#               36.89489499497731
#             ],
#             [
#               -121.83912992477416,
#               36.880204177187395
#             ]
#           ]
#         ]
#       }
#     }
#   ]
# }

coordinates = json_data['features'][0]['geometry']['coordinates']

lng, lat = np.mean(coordinates[0], axis=0) 
print((lng, lat))

(-122.5125961303711, 37.763630012936254)


Make a simple map and add your ROI

In [9]:
Map = geemap.Map(center=(lat, lng), zoom=14)
json_layer = GeoJSON(data=json_data, name='ROI', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
Map.add_layer(json_layer)

Map

Map(center=[37.763630012936254, -122.5125961303711], controls=(WidgetControl(options=['position', 'transparent…

## Sentinel2 (10-m R/G/B bands)

Make a Sentinel2-MSI image collection from a particular AOI and date range

In [5]:
OUT_RES_M = 10 #output pixel size in m (10m for sentinel visisble band)

gee_collection = 'COPERNICUS/S2_SR'

collection = ee.ImageCollection(gee_collection)

polys = ee.Geometry.Polygon(coordinates)
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
# print("lng = {}, lat = {}".format(lng, lat))

lng_lat = ee.Geometry.Point(lng, lat)
collection = collection.filterBounds(polys)
collection = collection.filterDate(start_date, end_date)
count = collection.size().getInfo()
print("Number of cloudy scenes: ", count)

collection = collection.filter('CLOUDY_PIXEL_PERCENTAGE < '+str(MAXCLOUD))

count = collection.size().getInfo()
print("Number of non-cloudy scenes: ", count)

Number of cloudy scenes:  302
Number of non-cloudy scenes:  93


Cycle through an get each scene name

In [6]:
img_lst = collection.toList(1000)

N = []
for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    N.append(name)

Cycle through and export

In [7]:
for n in N:
    print(n)
    image = ee.Image(gee_collection+'/'+n).select(['B4', 'B3', 'B2'])
    try:
        os.remove(os.getcwd()+os.sep+n+'.tif')
    except:
        pass
    geemap.ee_export_image(image, os.getcwd()+os.sep+n+'.tif', scale=OUT_RES_M, region=polys) #, file_per_band=True)
    
    image = ee.Image(gee_collection+'/'+n).select(['B8'])
    try:
        os.remove(os.getcwd()+os.sep+n+'_ir.tif')
    except:
        pass
    geemap.ee_export_image(image, os.getcwd()+os.sep+n+'_ir.tif', scale=OUT_RES_M, region=polys) #, file_per_band=True)
    
    

20181231T185801_20181231T190045_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20181231T185801_20181231T190045_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20181231T185801_20181231T190045_T10SEF_ir.tif
20181231T185801_20181231T190045_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20181231T185801_20181231T190045_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20181231T185801_20181231T190045_T10SFF_ir.tif
20190125T185649_20190125T185958_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190125T185649_20190125T185958_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatS

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190331T184941_20190331T185952_T10SEF_ir.tif
20190331T184941_20190331T185952_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190331T184941_20190331T185952_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190331T184941_20190331T185952_T10SFF_ir.tif
20190410T184921_20190410T185357_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190410T184921_20190410T185357_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190410T184921_20190410T185357_T10SEF_ir.tif
20190410T184921_20190410T185357_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satsho

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190912T184949_20190912T190314_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190912T184949_20190912T190314_T10SFF_ir.tif
20190917T185021_20190917T185918_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190917T185021_20190917T185918_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190917T185021_20190917T185918_T10SEF_ir.tif
20190917T185021_20190917T185918_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190917T185021_20190917T185918_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20190917T185021_20190917T18591

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20191022T185429_20191022T185427_T10SEF_ir.tif
20191022T185429_20191022T185427_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20191022T185429_20191022T185427_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20191022T185429_20191022T185427_T10SFF_ir.tif
20191027T185501_20191027T190301_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20191027T185501_20191027T190301_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20191027T185501_20191027T190301_T10SEF_ir.tif
20191101T185529_20191101T185712_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satsho

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200209T185529_20200209T190321_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200209T185529_20200209T190321_T10SEF_ir.tif
20200219T185419_20200219T190153_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200219T185419_20200219T190153_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200219T185419_20200219T190153_T10SEF_ir.tif
20200219T185419_20200219T190153_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200219T185419_20200219T190153_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200219T185419_20200219T19015

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200504T184921_20200504T185732_T10SFF_ir.tif
20200524T184921_20200524T185506_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200524T184921_20200524T185506_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200524T184921_20200524T185506_T10SEF_ir.tif
20200524T184921_20200524T185506_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200524T184921_20200524T185506_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200524T184921_20200524T185506_T10SFF_ir.tif
20200603T184921_20200603T185923_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satsho

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200807T184919_20200807T185448_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200807T184919_20200807T185448_T10SFF_ir.tif
20200812T184921_20200812T185701_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200812T184921_20200812T185701_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200812T184921_20200812T185701_T10SFF_ir.tif
20200906T184919_20200906T185437_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200906T184919_20200906T185437_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20200906T184919_20200906T18543

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201016T185359_20201016T185722_T10SFF_ir.tif
20201021T185431_20201021T190300_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201021T185431_20201021T190300_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201021T185431_20201021T190300_T10SFF_ir.tif
20201026T185459_20201026T190015_T10SEF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201026T185459_20201026T190015_T10SEF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201026T185459_20201026T190015_T10SEF_ir.tif
20201026T185459_20201026T190015_T10SFF
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satsho

Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201220T185811_20201220T190036_T10SFF.tif
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/20201220T185811_20201220T190036_T10SFF_ir.tif


In [ ]:
# for n in N:
#     try:
#         os.remove(n+'.tif')
#     except:
#         pass
#     cmd = 'gdal_merge.py -separate -o '+n+'.tif -co PHOTOMETRIC=MINISBLACK '+n+'.B4.tif '+n+'.B3.tif '+n+'.B2.tif'
#     os.system(cmd)

In [ ]:
# for file in *.tif; do gdal_translate -of JPEG -scale -co worldfile=yes $file "${file%.tif}.jpg"; done

In [ ]:
#convert -delay 20 -loop 0 -resize 96x408 *.jpg sentinel2_OB_20181231_20201220.gif

## Landsat-7 (15-m pansharpened R/G/B)

USGS Landsat 7 Collection 1 Tier 1 TOA Reflectance

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_TOA

In [8]:
# gee_collection = 'LANDSAT/LE07/C01/T1_TOA' 

# OUT_RES_M = 15

# collection = ee.ImageCollection(gee_collection)

# polys = ee.Geometry.Polygon(coordinates)
# centroid = polys.centroid()
# lng, lat = centroid.getInfo()['coordinates']
# # print("lng = {}, lat = {}".format(lng, lat))

# lng_lat = ee.Geometry.Point(lng, lat)
# collection = collection.filterBounds(polys)
# collection = collection.filterDate(start_date, end_date).sort('system:time_start', True) 
# count = collection.size().getInfo()
# print("Number of cloudy scenes: ", count)


Number of cloudy scenes:  368


In [ ]:
# img_lst = collection.toList(1000)

# N = []
# for i in range(0, count):
#     image = ee.Image(img_lst.get(i))
#     #cloudscore = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
#     #print(cloudscore)
#     name = image.get('system:index').getInfo()
#     N.append(name)

In [22]:
# for n in N:
#     print(n)
#     image = ee.Image(gee_collection+'/'+n)
#     hsv = image.select(['B3', 'B2', 'B1']).rgbToHsv()
#     sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), image.select('B8')]).hsvToRgb()    
#     try:
#         os.remove(os.getcwd()+os.sep+n+'.tif')
#     except:
#         pass
#     geemap.ee_export_image(sharpened,os.getcwd()+os.sep+n+'.tif', scale=OUT_RES_M, region=polys) #, file_per_band=True)

LE07_044034_20150108
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20150108.tif
LE07_045034_20150115
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20150115.tif
LE07_044034_20150124
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20150124.tif
LE07_045034_20150131
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20150131.tif
LE07_044034_20150209
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20150209.tif
LE07_045034_20150216
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20150216.tif
LE07_044034_20150225
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20150225.tif
LE07_045034_20150304
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20150304.tif
LE07_044034_20150313
Gen

Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20151023.tif
LE07_045034_20151030
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20151030.tif
LE07_044034_20151108
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20151108.tif
LE07_045034_20151115
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20151115.tif
LE07_044034_20151124
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20151124.tif
LE07_045034_20151201
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20151201.tif
LE07_044034_20151210
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20151210.tif
LE07_044034_20151226
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20151226.tif
LE07_044034_20160111
Generating URL ...
Please wait ...
Data dow

Data downloaded to /home/marda/Downloads/LE07_045034_20160728.tif
LE07_044034_20160806
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20160806.tif
LE07_045034_20160813
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20160813.tif
LE07_044034_20160822
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20160822.tif
LE07_045034_20160829
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20160829.tif
LE07_044034_20160907
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20160907.tif
LE07_045034_20160914
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20160914.tif
LE07_044034_20160923
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20160923.tif
LE07_045034_20160930
Generating URL ...
Please wait ...
Data downloaded to /home

Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20170606.tif
LE07_045034_20170613
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20170613.tif
LE07_044034_20170622
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20170622.tif
LE07_045034_20170629
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20170629.tif
LE07_044034_20170708
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20170708.tif
LE07_045034_20170715
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20170715.tif
LE07_044034_20170724
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20170724.tif
LE07_045034_20170731
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20170731.tif
LE07_044034_20170809
Generating URL ...
Please wait ...
Data dow

Data downloaded to /home/marda/Downloads/LE07_044034_20180201.tif
LE07_045034_20180208
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20180208.tif
LE07_044034_20180217
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20180217.tif
LE07_045034_20180224
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20180224.tif
LE07_044034_20180305
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20180305.tif
LE07_045034_20180328
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20180328.tif
LE07_045034_20180413
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20180413.tif
LE07_044034_20180422
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20180422.tif
LE07_044034_20180508
Generating URL ...
Please wait ...
Data downloaded to /home

Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20181031.tif
LE07_045034_20181107
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20181107.tif
LE07_044034_20181116
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20181116.tif
LE07_044034_20181202
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20181202.tif
LE07_044034_20181218
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20181218.tif
LE07_045034_20181225
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20181225.tif
LE07_044034_20190103
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20190103.tif
LE07_044034_20190119
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20190119.tif
LE07_045034_20190126
Generating URL ...
Please wait ...
Data dow

Data downloaded to /home/marda/Downloads/LE07_044034_20190815.tif
LE07_045034_20190822
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20190822.tif
LE07_044034_20190831
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20190831.tif
LE07_044034_20190916
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20190916.tif
LE07_045034_20190923
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20190923.tif
LE07_044034_20191002
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20191002.tif
LE07_045034_20191009
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20191009.tif
LE07_044034_20191018
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20191018.tif
LE07_045034_20191025
Generating URL ...
Please wait ...
Data downloaded to /home

Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20200614.tif
LE07_045034_20200621
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20200621.tif
LE07_044034_20200630
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20200630.tif
LE07_045034_20200707
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20200707.tif
LE07_044034_20200716
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20200716.tif
LE07_045034_20200723
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20200723.tif
LE07_044034_20200801
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_044034_20200801.tif
LE07_045034_20200808
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/LE07_045034_20200808.tif
LE07_044034_20200817
Generating URL ...
Please wait ...
Data dow

In [ ]:
#!for file in *.tif; do gdal_translate -of JPEG -scale -co worldfile=yes $file "${file%.tif}.jpg"; done

In [ ]:
#convert -delay 20 -loop 0 -resize 96x408 *.jpg landsat7_OB_20150124_20201223.gif

## Landsat-8 (15-m pansharpened R/G/B)

USGS Landsat 8 Collection 1 Tier 1 TOA Reflectance

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_TOA

don't know how to implement cloud filtering ...

In [6]:
# def cloudMaskL457(image):
#   qa = image.select('pixel_qa')
#   # If the cloud bit (5) is set and the cloud confidence (7) is high
#   # or the cloud shadow bit is set (3), then it's a bad pixel.
#   cloud = qa.bitwiseAnd(1 << 5) \
#           .And(qa.bitwiseAnd(1 << 7)) \
#           .Or(qa.bitwiseAnd(1 << 3))
#   # Remove edge pixels that don't occur in all bands
#   mask2 = image.mask().reduce(ee.Reducer.min())
#   return image.updateMask(cloud.Not()).updateMask(mask2)

In [10]:
gee_collection = 'LANDSAT/LC08/C01/T1_TOA' 

OUT_RES_M = 15

# MAXCLOUD = 5

# def cloudMask(img):
#     cloudscore = ee.Algorithms.Landsat.simpleCloudScore(img).select('cloud')
#     return img.updateMask(cloudscore.lt(MAXCLOUD))

collection = ee.ImageCollection(gee_collection)

polys = ee.Geometry.Polygon(coordinates)
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
# print("lng = {}, lat = {}".format(lng, lat))

lng_lat = ee.Geometry.Point(lng, lat)
collection = collection.filterBounds(polys)
collection = collection.filterDate(start_date, end_date).sort('system:time_start', True) 
count = collection.size().getInfo()
print("Number of cloudy scenes: ", count)

# collection = collection.filter('CLOUDY_PIXEL_PERCENTAGE < '+str(MAXCLOUD))

# collection = collection.map(cloudMaskL457).sort('system:time_start', True) 

# count = collection.size().getInfo()
# print("Number of non-cloudy scenes: ", count)

Number of cloudy scenes:  245


In [11]:
img_lst = collection.toList(1000)

N = []
for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    #cloudscore = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
    #print(cloudscore)
    name = image.get('system:index').getInfo()
    N.append(name)

Pansharpen to 15-m and export directly as RGB

In [17]:
for n in N:
    print(n)
    image = ee.Image(gee_collection+'/'+n)
    hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()
    sharpened = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), image.select('B8')]).hsvToRgb()    
    try:
        os.remove(os.getcwd()+os.sep+n+'.tif')
    except:
        pass
    geemap.ee_export_image(sharpened,os.getcwd()+os.sep+n+'.tif', scale=OUT_RES_M, region=polys) #, file_per_band=True)

    hsv = sharpened.select(['B4', 'B3', 'B2']).rgbToHsv()
    sharpened_ir = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), image.select('B5')]).hsvToRgb()    

    try:
        os.remove(os.getcwd()+os.sep+n+'_ir.tif')
    except:
        pass
    geemap.ee_export_image(sharpened_ir, os.getcwd()+os.sep+n+'_ir.tif', scale=OUT_RES_M, region=polys, file_per_band=True)
        

LC08_045034_20150107
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/LC08_045034_20150107.tif
Generating URL ...
An error occurred while downloading.
Image.select: Pattern 'B4' did not match any bands.
LC08_044034_20150116
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/LC08_044034_20150116.tif
Generating URL ...
An error occurred while downloading.
Image.select: Pattern 'B4' did not match any bands.
LC08_045034_20150123
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/LC08_045034_20150123.tif
Generating URL ...
An error occurred while downloading.
Image.select: Pattern 'B4' did not match any bands.
LC08_044034_20150201
Generating URL ...
Please wait ...
Data downloaded to /media/marda/TWOTB/USGS/DATA/SatShorelines/satshoreline_unet/LC08_044034_20150201.tif
Generating URL ...
An error occu

KeyboardInterrupt: 

In [25]:
ee.ee_list.List(hsv.bandNames())

In [24]:
g.contains

<bound method ApiFunction.importApi.<locals>.MakeBoundFunction.<locals>.<lambda> of <ee.ee_list.List object at 0x7f3c5eb03430>>

convert to jpeg with world file, for use in doodler (bash assuming gdal_translate is installed and on system path)

In [ ]:
# !for file in *.tif; do gdal_translate -of JPEG -scale -co worldfile=yes $file "${file%.tif}.jpg"; done

In [ ]:
# after I sorted the bad ones out, I ran this to make an animated gif:
# convert -delay 20 -loop 0 -resize 96x408 *.jpg landsat8_OB_20150201_20201120.gif

## NAIP (1-m RGB)

In [12]:
gee_collection = 'USDA/NAIP/DOQQ' 
OUT_RES_M = 1

In [13]:
collection = ee.ImageCollection(gee_collection)

polys = ee.Geometry.Polygon(coordinates)
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
# print("lng = {}, lat = {}".format(lng, lat))

lng_lat = ee.Geometry.Point(lng, lat)
collection = collection.filterBounds(polys)
collection = collection.filterDate(start_date, end_date).sort('system:time_start', True) 
count = collection.size().getInfo()
print("Number of cloudy scenes: ", count)

Number of cloudy scenes:  2


In [14]:
img_lst = collection.toList(1000)

N = []
for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    N.append(name)

In [15]:
for n in N:
        image = ee.Image('USDA/NAIP/DOQQ/'+n)
        Map.addLayer(image, {'bands': ['R', 'G','B']}, 'NAIP')
        geemap.ee_export_image(image, os.getcwd()+os.sep+n+'.tif', scale=OUT_RES_M, region=polys)

Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/m_3712212_se_10_h_20160625.tif
Generating URL ...
Please wait ...
Data downloaded to /home/marda/Downloads/m_3712212_se_10_060_20180804_20190209.tif
